<i>
Student Name: <b>Priyanshi Agrawal</b> <br>
Student Number: <b>25205515</b> <br>
Module Code: <b>COMP47670</b> <br>
Assignment 1: Autumn 2025 <br>
Dataset Selected: <b>Used Cars for Sale</b>
</i>
<hr>

The notebook is based on Data Collection from the given website - http://mlg.ucd.ie/modules/python/assign1/ <br>
I have choosen <b>'3. Used Cars for Sale'</b> Database - http://mlg.ucd.ie/modules/python/assign1/cars/index.html
<hr>

Import all modules and define base url for the database. <br>
Define a method to get soup object for the url passed.

In [9]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv

baseUrl='http://mlg.ucd.ie/modules/python/assign1/cars/'
indexUrl=baseUrl+'index.html' #first page of database

def get_soup(url):
    html=urlopen(url)
    soup=BeautifulSoup(html, 'lxml')
    return soup

indexSoup=get_soup(indexUrl) #get soup oject to parse the html

Create a Dictionary to store all urls for each manufacturer: [key,value]=[iindex,list(url)] <br>
The loop below iterates over list of <a> tages and stores base url in dictiionary for each manufacturer page.

In [10]:
manufacturerDict={} #this dictionary stores (key,value)=(index,list of all. url) for each manufacturer
count=1
for link in indexSoup.find_all('a'):
    manufacturerDict[count]=[] #initiallize list
    manufacturerDict[count].append(baseUrl+link.get('href')) #add base url for each manufacturer
    count+=1

Define function to get all pages url for each manufacturer as it contains pagination.

In [11]:
def get_max_page(soup):
    list=soup.find_all('a') #find all <a> tag in each manufacturer base url passed
    pages=[]
    for a in list:
        pages.append(baseUrl+a.get('href')) #get href attribute value and store in list as a valid url
    size=len(pages)
    return pages[1:size-1] #pages[0]=index.html and pages[size-1]=nextpage link, skip both url

Extract all page-urls into the dictionary with below loop

In [12]:
for url in manufacturerDict:
    pageList = get_max_page(get_soup(manufacturerDict[url][0])) #pass baseurl for each manufacturer
    manufacturerDict[url].extend(pageList) #extend the list with all page urls

Define a function to collect data of all cars present on a single url/page. <br>
<b>Clean and transform price and mileage fields in the function itself.</b>

In [13]:
def get_page_data(url, soup):
    carSoup=soup.find_all('li', {'class':'car-item'}) #list of all car-card
    pageData=[] #store all car data
    for car in carSoup:
        data=[] #store single car data
        model=car.find('h3', {'class':'make-model'}).get_text(strip=True)
        data.append(model.lower())
        rows=car.find_all('tr')
        count=0
        for row in rows:
            td=row.find_all('td')[1].get_text(strip=True).lower() #get only field value
            if count==0:
                td=td.replace(',', '').replace(' ', '').replace('€','') #clean price field
            elif count==3:
                td=td.replace(',','').replace(' ', '').replace('miles','') #clean mileage field
            count+=1
            data.append(td) #add each field value for a single car
        pageData.append(data) #add list of values for each car
    return pageData #2d list containis all values for a page

Define a function to convert 2D list into csv file for each filename passed

In [14]:
def data_to_csv(fileName, fields, data):
    file=open(fileName, 'w') #open file in 'write' mode, if file does not exist, it gets created.
    writer = csv.writer(file)
    writer.writerow(fields) #write field names
    writer.writerows(data) #write field data
    file.close() #close file after use

Define a list of fields/column name 

In [15]:
fieldName=["Name",
           "SalePrice",
           "DateOfSale",
           "Year","Mileage",
           "Classification",
           "Transmission","FuelType",
           "Description",
           "SaleLocation"]

Final step, get all data from all pages and save into csv files for each manufacturer.

In [16]:
for key in manufacturerDict:
    carList=[]
    for url in manufacturerDict[key]: #for each key, get car data for all pages/url
        pageSoup=get_soup(url)
        pageData=get_page_data(url, pageSoup)
        carList.extend(pageData) #append all car data from each page
    fileName=f'{key}.csv' 
    data_to_csv(fileName, fieldName, carList) #save data in. csv file

In [17]:
print("all data saved in csv")

all data saved in csv
